In [12]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [13]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from glasses.nn.models.classification.efficientnet import SEInvertedResidualBlock, EfficientNetEncoder


SEInvertedResidualBlock(32, 64)

SEInvertedResidualBlock(
  (block): Residual(
    (block): Sequential(
      (exp): ConvBnAct(
        (conv): Conv2dPad(32, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): Swish()
      )
      (conv): Sequential(
        (0): ConvBnAct(
          (conv): DepthWiseConv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=192, bias=False)
          (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): Swish()
        )
        (1): Sequential(
          (0): SEModuleConv(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (att): Sequential(
              (0): Conv2d(192, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): Swish()
              (2): Conv2d(1, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (3): Sigmoid()
            )
          )
          

In [5]:
EfficientNetEncoder()

EfficientNetEncoder(
  (gate): Sequential(
    (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blocks): ModuleList(
    (0): EfficientNetLayer(
      (block): Sequential(
        (0): SEInvertedResidualBlock(
          (block): Residual(
            (block): Sequential(
              (conv): Sequential(
                (0): ConvBnAct(
                  (conv): DepthWiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (act): Swish()
                )
                (1): Sequential(
                  (0): SEModuleConv(
                    (avg_pool): AdaptiveAvgPool2d(output_size=1)
                    (att): Sequential(
                      (0): Conv2d(32, 1, kernel_size=(1, 1), stride=(1, 1

In [17]:
from efficientnet_pytorch import EfficientNet
from torchsummary import summary


model = EfficientNet.from_pretrained('efficientnet-b4')

summary(model.cuda(), (3,224,224))

[autoreload of glasses.nn.models.classification.efficientnet failed: Traceback (most recent call last):
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line


Loaded pretrained weights for efficientnet-b4
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 226, 226]               0
Conv2dStaticSamePadding-2         [-1, 48, 112, 112]           1,296
       BatchNorm2d-3         [-1, 48, 112, 112]              96
MemoryEfficientSwish-4         [-1, 48, 112, 112]               0
         ZeroPad2d-5         [-1, 48, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 48, 112, 112]             432
       BatchNorm2d-7         [-1, 48, 112, 112]              96
MemoryEfficientSwish-8         [-1, 48, 112, 112]               0
          Identity-9             [-1, 48, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 12, 1, 1]             588
MemoryEfficientSwish-11             [-1, 12, 1, 1]               0
         Identity-12             [-1, 12, 1, 1]               0
Conv2dStaticSamePadding-13       

In [18]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [25]:
from glasses.nn.models.classification.efficientnet import EfficientNet

model  = EfficientNet()


summary(model.cuda(), (3,224,224))

[(32, 16), (16, 24), (24, 40), (40, 80), (80, 112), (112, 192), (192, 320)]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
   DepthWiseConv2d-3         [-1, 32, 112, 112]             288
       BatchNorm2d-4         [-1, 32, 112, 112]              64
             Swish-5         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-6             [-1, 32, 1, 1]               0
            Conv2d-7              [-1, 1, 1, 1]              32
             Swish-8              [-1, 1, 1, 1]               0
            Conv2d-9             [-1, 32, 1, 1]              32
          Sigmoid-10             [-1, 32, 1, 1]               0
     SEModuleConv-11         [-1, 32, 112, 112]               0
        Conv2dPad-12         [-1, 16, 112, 112]             512
      BatchNorm2d-13       

[autoreload of glasses.nn.models.classification.efficientnet failed: Traceback (most recent call last):
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autorelo

In [24]:
model

EfficientNet(
  (encoder): EfficientNetEncoder(
    (gate): Sequential(
      (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (blocks): ModuleList(
      (0): EfficientNetLayer(
        (block): Sequential(
          (0): SEInvertedResidualBlock(
            (block): Residual(
              (block): Sequential(
                (conv): Sequential(
                  (0): ConvBnAct(
                    (conv): DepthWiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                    (act): Swish()
                  )
                  (1): Sequential(
                    (0): SEModuleConv(
                      (avg_pool): AdaptiveAvgPool2d(output_size=1)
                      (att): Sequential(
      